In [41]:
# setup
!pip install datasets pandas mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 7.9 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:00m eta 0:00:010:01:01
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0


In [42]:
# load the data
from datasets import load_dataset
import pandas as pd
dataset = load_dataset('isavita/advent-of-code', data_files='train.json', split='train')
df = pd.DataFrame(dataset)
print(dataset)

Dataset({
    features: ['answer', 'input', 'name', 'solution', 'solution_lang', 'task', 'year'],
    num_rows: 7861
})


In [43]:
# get solutions in python
df_python = df[df['solution_lang'] == 'python']
print(df_python.info())

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 30 to 7856
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   answer         418 non-null    object
 1   input          418 non-null    object
 2   name           418 non-null    object
 3   solution       418 non-null    object
 4   solution_lang  418 non-null    object
 5   task           418 non-null    object
 6   year           418 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 26.1+ KB
None


In [106]:
user_prefix = """
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
The assistant can interact with an interactive Python (Jupyter Notebook) environment and receive the corresponding output when needed. The code should be enclosed using "<execute_ipython>" tag, for example:
<execute_ipython>
print("Hello World!")
</execute_ipython>
The assistant can execute bash commands on behalf of the user by wrapping them with <execute_bash> and </execute_bash>.
For example, you can list the files in the current directory by <execute_bash> ls </execute_bash>.

Apart from the standard Python library, the assistant can also use the following functions (already imported) in <execute_ipython> environment:
open_file(path: str, line_number: Optional[int] = None) -> None:
    Opens the file at the given path in the editor. If line_number is provided, the window will be moved to include that line.
    Args:
    path: str: The path to the file to open.
    line_number: Optional[int]: The line number to move to.

goto_line(line_number: int) -> None:
    Moves the window to show the specified line number.
    Args:
    line_number: int: The line number to move to.

scroll_down() -> None:
    Moves the window down by 100 lines.
    Args:
    None

scroll_up() -> None:
    Moves the window up by 100 lines.
    Args:
    None

create_file(filename: str) -> None:
    Creates and opens a new file with the given name.
    Args:
    filename: str: The name of the file to create.

append_file(file_name: str, content: str) -> None:
    Append content to the given file.
    It appends text `content` to the end of the specified file.
    Args:
    file_name: str: The name of the file to append to.
    content: str: The content to append to the file.

edit_file(file_name: str, start: int, end: int, content: str) -> None:
    Edit a file.
    Replaces in given file `file_name` the lines `start` through `end` (inclusive) with the given text `content`.
    Args:
    file_name: str: The name of the file to edit.
    start: int: The start line number. Must satisfy start >= 1.
    end: int: The end line number. Must satisfy start <= end <= number of lines in the file.
    content: str: The content to replace the lines with.

search_dir(search_term: str, dir_path: str = './') -> None:
    Searches for search_term in all files in dir. If dir is not provided, searches in the current directory.
    Args:
    search_term: str: The term to search for.
    dir_path: Optional[str]: The path to the directory to search.

search_file(search_term: str, file_path: Optional[str] = None) -> None:
    Searches for search_term in file. If file is not provided, searches in the current open file.
    Args:
    search_term: str: The term to search for.
    file_path: Optional[str]: The path to the file to search.

find_file(file_name: str, dir_path: str = './') -> None:
    Finds all files with the given name in the specified directory.
    Args:
    file_name: str: The name of the file to find.
    dir_path: Optional[str]: The path to the directory to search.

parse_pdf(file_path: str) -> None:
    Parses the content of a PDF file and prints it.
    Args:
    file_path: str: The path to the file to open.

parse_docx(file_path: str) -> None:
    Parses the content of a DOCX file and prints it.
    Args:
    file_path: str: The path to the file to open.

parse_latex(file_path: str) -> None:
    Parses the content of a LaTex file and prints it.
    Args:
    file_path: str: The path to the file to open.

parse_pptx(file_path: str) -> None:
    Parses the content of a pptx file and prints it.
    Args:
    file_path: str: The path to the file to open.

Please note that THE `edit_file` FUNCTION REQUIRES PROPER INDENTATION. If the assistant would like to add the line '        print(x)', it must fully write that out, with all those spaces before the code! Indentation is important and code that is not indented correctly will fail and require fixing before it can be run.

The assistant's response should be concise.
The assistant should include ONLY ONE <execute_ipython> or <execute_bash> in every one of the responses, unless the assistant is finished with the task or needs more input or action from the user in order to proceed.
IMPORTANT: Whenever possible, execute the code for the user using <execute_ipython> or <execute_bash> instead of providing it.

----------

Here is an example of how you can interact with the environment for task solving:

--- START OF EXAMPLE ---

USER: Please create a Python script that prints all even numbers from 1 to 10.

ASSISTANT: Let's start by creating a new file named `print_even_numbers.py`.
<execute_ipython>
create_file('print_even_numbers.py')
</execute_ipython>

USER:
OBSERVATION:
[File: print_even_numbers.py (0 lines total)]
[File print_even_numbers.py created.]

ASSISTANT: Now, let's add the code to print all even numbers from 1 to 10.
<execute_ipython>
CODE = \"\"\"for i in range(1, 11):
    if i % 2 == 0:
        print(i)
\"\"\"
edit_file('print_even_numbers.py', start=1, end=1, content=CODE)
</execute_ipython>

USER:
OBSERVATION:
[File: print_even_numbers.py (4 lines total)]
1|for i in range(1, 11):
2|    if i % 2 == 0:
3|        print(i)
[File updated. Please review the changes and make sure they are correct (correct indentation, no duplicate lines, etc). Edit the file again if necessary.]

--- END OF EXAMPLE ---

"""

user_task_prompt_template = """
{prefix}

Challenge for Advent of Code:
{task}

Please write the solution in {lang}. Save the solution in a file named solution.{lang_ext}. Ensure that your solution reads input from a file named input.txt in the same directory and prints the output to stdout.
"""

lang_ext_mapping = {
    'python': 'py',
    'javascript': 'js',
    'ruby': 'rb'
}

row = df_python.iloc[0]
# print(user_task_prompt_template.format(prefix=user_prefix, task=row['task'], lang=row['solution_lang'], lang_ext=lang_ext_mapping[row['solution_lang']]))

# prompt for creating a new empty file
assistant_create_file_prompt_template = """
Creating the {lang} file `solution.{lang_ext}`:
<execute_ipython>
create_file('solution.{lang_ext}')
</execute_ipython>
"""
# print(assistant_create_file_prompt_template.format(lang=row['solution_lang'], lang_ext=lang_ext_mapping[row['solution_lang']]))

# Prompt for observing the creation of a new empty file
user_observation_prompt_template = """
OBSERVATION:
[File: /workspace/solution.{lang_ext} (1 lines total)]
1|
[File solution.{lang_ext} created.]
"""
# print(user_observation_prompt_template.format(lang_ext=lang_ext_mapping[row['solution_lang']]))

# prompt for writing the solution to a file
assistant_edit_file_prompt_template = """
Writing the {lang} code to `solution.{lang_ext}`:
<execute_ipython>
EDITED_CODE=\"\"\"{solution}\"\"\"
edit_file('solution.{lang_ext}', start=1, end=1, content=EDITED_CODE)
</execute_ipython>
"""
print(assistant_edit_file_prompt_template.format(solution=row['solution'], lang=row['solution_lang'], lang_ext=lang_ext_mapping[row['solution_lang']]))


Writing the python code to `solution.py`:
<execute_ipython>
EDITED_CODE="""with open("input.txt", "r") as file:
    data = file.read()

floor = 0
for char in data:
    if char == "(":
        floor += 1
    elif char == ")":
        floor -= 1

print(floor)"""
edit_file('solution.py', start=1, end=1, content=EDITED_CODE)
</execute_ipython>



In [94]:
# generate python dataset solutions

df_python_formatted = [
    {
        "messages": [
            {
                "role": "user",
                "content": user_task_prompt_template.format(prefix=user_prefix, task=row['task'], lang=row['solution_lang'], lang_ext=lang_ext_mapping[row['solution_lang']])
            },
            {
                "role": "assistant", 
                "content": assistant_create_file_prompt_template.format(lang=row['solution_lang'], lang_ext=lang_ext_mapping[row['solution_lang']])
            },
            {
                "role": "user",
                "content": user_observation_prompt_template.format(lang_ext=lang_ext_mapping[row['solution_lang']])
            },
            {
                "role": "assistant", 
                "content": assistant_edit_file_prompt_template.format(solution=row['solution'], lang=row['solution_lang'], lang_ext=lang_ext_mapping[row['solution_lang']])
            },
        ]
    }
    for index, row in df_python.iterrows()
]
print(df_python_formatted[0])

{'messages': [{'role': 'user', 'content': '\n\nA chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user\'s questions.\nThe assistant can interact with an interactive Python (Jupyter Notebook) environment and receive the corresponding output when needed. The code should be enclosed using "<execute_ipython>" tag, for example:\n<execute_ipython>\nprint("Hello World!")\n</execute_ipython>\nThe assistant can execute bash commands on behalf of the user by wrapping them with <execute_bash> and </execute_bash>.\nFor example, you can list the files in the current directory by <execute_bash> ls </execute_bash>.\n\nApart from the standard Python library, the assistant can also use the following functions (already imported) in <execute_ipython> environment:\nopen_file(path: str, line_number: Optional[int] = None) -> None:\n    Opens the file at the given path in the editor. If line_number is provided, the window w

In [108]:
import json
import os

# ensure the directory exists
os.makedirs('../data', exist_ok=True)

# create the training data
data_path = f"../data/python_{len(df_python_formatted)}.jsonl"
with open(data_path, "w") as f:
    for line in df_python_formatted:
        json.dump(line, f)
        f.write("\n")

In [126]:
import os
import json
import random
from mistralai.client import MistralClient

# load the data
with open(data_path, "r") as f:
    data = f.readlines()

# shuffle the data
random.shuffle(data)

# calculate the split point
split_point = int(0.97 * len(data))

# split the data
train_data = data[:split_point]
eval_data = data[split_point:]

print(f'Training data size: {len(train_data)}')
print(f'Evaluation data size: {len(eval_data)}')

# write training data to a new file
with open("../data/chunk_train.jsonl", "w") as f:
    for line in train_data:
        f.write(line)

# write evaluation data to a new file
with open("../data/chunk_eval.jsonl", "w") as f:
    for line in eval_data:
        f.write(line)

# initialize Mistral client
api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

# upload the training file
with open("../data/chunk_train.jsonl", "rb") as f:
    chunk_train = client.files.create(file=("chunk_train.jsonl", f))

# upload the evaluation file
with open("../data/chunk_eval.jsonl", "rb") as f:
    chunk_eval = client.files.create(file=("chunk_eval.jsonl", f))

Training data size: 405
Evaluation data size: 13


In [127]:
print(f'Upload training data:\n{chunk_train}')
print(f'Upload evaluation data:\n{chunk_eval}')

Upload training data:
id='61b7dd90-d8ea-4687-a095-c9f2834bdc02' object='file' bytes=4690232 created_at=1718539147 filename='chunk_train.jsonl' purpose='fine-tune'
Upload evaluation data:
id='f07d0e27-5e9c-47e7-9cac-b810d2722d83' object='file' bytes=147846 created_at=1718539148 filename='chunk_eval.jsonl' purpose='fine-tune'


In [131]:
from mistralai.models.jobs import TrainingParameters

# create training job
created_jobs = client.jobs.create(
    model="mistral-small-latest",
    training_files=[chunk_train.id],
    validation_files=[chunk_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=10,
        learning_rate=0.0001,
        )
)
print(created_jobs)

id='b637638d-20c4-44a8-afdf-746b17e18cfd' hyperparameters=TrainingParameters(training_steps=100, learning_rate=0.0001) fine_tuned_model=None model='mistral-small-latest' status='QUEUED' job_type='FT' created_at=1718540979 modified_at=1718540980 training_files=['61b7dd90-d8ea-4687-a095-c9f2834bdc02'] validation_files=['f07d0e27-5e9c-47e7-9cac-b810d2722d83'] object='job' integrations=[]


In [144]:
# retrieve a jobs
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
print(retrieved_jobs)

id='b637638d-20c4-44a8-afdf-746b17e18cfd' hyperparameters=TrainingParameters(training_steps=100, learning_rate=0.0001) fine_tuned_model='ft:mistral-small-latest:8aad839f:20240616:b637638d' model='mistral-small-latest' status='SUCCESS' job_type='FT' created_at=1718540979 modified_at=1718542199 training_files=['61b7dd90-d8ea-4687-a095-c9f2834bdc02'] validation_files=['f07d0e27-5e9c-47e7-9cac-b810d2722d83'] object='job' integrations=[] events=[Event(name='status-updated', data={'status': 'SUCCESS'}, created_at=1718542199), Event(name='status-updated', data={'status': 'RUNNING'}, created_at=1718540984), Event(name='status-updated', data={'status': 'QUEUED'}, created_at=1718540980)] checkpoints=[Checkpoint(metrics=Metric(train_loss=0.058877, valid_loss=0.254303, valid_mean_token_accuracy=None), step_number=100, created_at=1718542183), Checkpoint(metrics=Metric(train_loss=0.092092, valid_loss=0.253881, valid_mean_token_accuracy=1.19241), step_number=80, created_at=1718541957), Checkpoint(met

In [154]:
from mistralai.models.chat_completion import ChatMessage
print(retrieved_jobs.fine_tuned_model)
prompt = """
Write me a python program that solves the following advent of code challenge:
--- Day 2: I Was Told There Would Be No Math ---
The elves are running low on wrapping paper, and so they need to submit an order for more. They have a list of the dimensions (length l, width w, and height h) of each present, and only want to order exactly as much as they need.

Fortunately, every present is a box (a perfect right rectangular prism), which makes calculating the required wrapping paper for each gift a little easier: find the surface area of the box, which is 2*l*w + 2*w*h + 2*h*l. The elves also need a little extra paper for each present: the area of the smallest side.

For example:

A present with dimensions 2x3x4 requires 2*6 + 2*12 + 2*8 = 52 square feet of wrapping paper plus 6 square feet of slack, for a total of 58 square feet.
A present with dimensions 1x1x10 requires 2*1 + 2*10 + 2*10 = 42 square feet of wrapping paper plus 1 square foot of slack, for a total of 43 square feet.
All numbers in the elves' list are in feet. How many total square feet of wrapping paper should they order?

The program should read it's input from file input.txt
"""
chat_response = client.chat(
    model=retrieved_jobs.fine_tuned_model,
    messages=[ChatMessage(role='user', content=prompt)]
)
print(chat_response.choices[0].message.content)

ft:mistral-small-latest:8aad839f:20240616:b637638d

Creating the python file `solution.py`:
<execute_ipython>
create_file('solution.py')
</execute_ipython>

